In [9]:
import yaml
import json
import vocab_build_tools as tools

stream = open('./config.yaml', 'r')

documents = yaml.load_all(stream, Loader=yaml.FullLoader)

for doc in documents:
    merged_df = tools.create_df(doc['termLists'])
    config = doc

examples = merged_df[~merged_df['examples'].isna()]

classes = ['TaxonConcept', 'TaxonConceptMappings', 'TaxonName', 'NomenclaturalType']

with open('../docs/examples/README.md', 'w') as f:
    
    f.write("# Examples\n\n")
    
    for cl in classes:
        f.write("\n## " + cl + "\n\n")
        
        for index, row in examples[examples['organizedInClass'] == 'http://rs.tdwg.org/tcs/terms/' + cl].iterrows():
            alias = [n['vann_preferredNamespacePrefix'] for n in config['termLists'] if n['vann_preferredNamespaceUri'] == row['namespace']][0]
            
            if type(row['examples']) == list:
                f.write("\n### " + alias + ":" + row['localName'] + "\n\n")
                for ex in row['examples']:
                    f.write(f"- [{ex}]({ex}.md)\n")
                    # text = tools.add_example(ex)
                    # f.write(text)


In [1]:
import yaml
import json
import vocab_build_tools as tools

stream = open('./config.yaml', 'r')

documents = yaml.load_all(stream, Loader=yaml.FullLoader)

for doc in documents:
    merged_df = tools.create_df(doc['termLists'])
    config = doc

examples = merged_df[~merged_df['examples'].isna()]

classes = ['TaxonConcept', 'TaxonConceptMappings', 'TaxonName', 'NomenclaturalType']

for cl in classes:
    for index, row in examples[examples['organizedInClass'] == 'http://rs.tdwg.org/tcs/terms/' + cl].iterrows():
        alias = [n['vann_preferredNamespacePrefix'] for n in config['termLists'] if n['vann_preferredNamespaceUri'] == row['namespace']][0]
        
        if type(row['examples']) == list:
            for ex in row['examples']:
                with open('../docs/examples/' + ex + '.md', 'w') as f:
                    heading = ex.replace('-', ' ')
                    f.write('# ' + heading + '\n')
                    text = tools.add_example(ex)
                    f.write(text)


In [17]:
import yaml
import vocab_build_tools as tools

stream = open('./config.yaml', 'r')

documents = yaml.load_all(stream, Loader=yaml.FullLoader)

for doc in documents:
    merged_df = tools.create_df(doc['termLists'])
    config = doc

examples = merged_df[~merged_df['examples'].isna()]

namespaces = []
aliases = []
for row in examples['examples'].to_list():
    if type(row) == list:
        for ex in row:
            path = '../examples/' + ex + '.ttl'
            with open(path, 'r') as f:
                for line in f:
                    if line[:7] == '@prefix':
                        ns = line[8:-2]
                        alias, uri = ns.split(': ', 1)
                        if alias not in aliases:
                            aliases.append(alias)
                            namespaces.append({
                                'alias': alias,
                                'uri': uri.strip(' <>')
                            })
                            
table = '| alias | uri |\n'
table += '|-|-|\n'
for ns in namespaces:
    table += '| ' + ns['alias'] + ' | ' + ns['uri'] + ' |\n'
    
print(table)

| alias | uri |
|-|-|
| tcs | http://rs.tdwg.org/tcs/terms/ |
| dcterms | http://purl.org/dc/terms/ |
| dwc | http://rs.tdwg.org/dwc/terms/ |
| bibo | http://purl.org/ontology/bibo/ |
| skosxl | http://www.w3.org/2008/05/skos-xl# |
| foaf | http://xmlns.com/foaf/0.1/ |
| oa | http://www.w3.org/ns/oa# |
| rdf | http://www.w3.org/1999/02/22-rdf-syntax-ns# |
| address | http://schemas.talis.com/2005/address/schema# |
| gbif | http://rs.gbif.org/terms/1.0/ |
| rdfs | http://www.w3.org/2000/01/rdf-schema# |

